In [3]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import random
import scipy
import glob
import os
from sklearn.metrics import root_mean_squared_error,r2_score
import seaborn as sns

In [10]:
CIs = []

def import_dfs():
    # Path to the folder containing CSV files
    folder_path = '../dados/'

    # Get a list of all CSV files in the folder
    csv_files = glob.glob(os.path.join(folder_path, '*.csv'))
    # Initialize an empty list to hold dataframes
    data_frames = []

    # Read the first CSV file to establish the schema
    schema = ['ester_mm', 'amox_mm', 'apa_mm', 'aoh_mm','apa_t']


    # Load each remaining CSV file, reorder columns, and append to the list
    for file in csv_files:
        df = pd.read_csv(file)  
        print(file)
        df = df[schema]  # Reorder columns to match the schema
        data_frames.append(df)
        CIs.append(df.iloc[0,:4].to_numpy())
    csv_files = [name.split('\\')[-1].rstrip('.csv') for name in csv_files]
    return data_frames,csv_files 
data_frames,file_list = import_dfs()
print(data_frames[0].columns)
print(file_list)
print(CIs)

../dados\25.45nh60ab20.csv
../dados\25.46nh30ab30.csv
../dados\25.47nh5ab80.csv
../dados\5.100nh40ab80.csv
../dados\5.102nh20ab40.csv
../dados\5.103nh40ab30.csv
../dados\5.106nh100ab40.csv
../dados\5.107nh55ab55.csv
../dados\5.109nh5ab55.csv
../dados\5.110nh10ab55.csv
../dados\5.112nh78ab35.csv
../dados\5.48nh20ab80.csv
../dados\5.80nh12ab40.csv
../dados\5.81nh5ab30.csv
../dados\nh60ab80.csv
Index(['ester_mm', 'amox_mm', 'apa_mm', 'aoh_mm', 'apa_t'], dtype='object')
['25.45nh60ab20', '25.46nh30ab30', '25.47nh5ab80', '5.100nh40ab80', '5.102nh20ab40', '5.103nh40ab30', '5.106nh100ab40', '5.107nh55ab55', '5.109nh5ab55', '5.110nh10ab55', '5.112nh78ab35', '5.48nh20ab80', '5.80nh12ab40', '5.81nh5ab30', 'nh60ab80']
[array([20.,  0., 60.,  0.]), array([30.,  0., 30.,  0.]), array([80.,  0.,  5.,  0.]), array([80.,  0., 40.,  0.]), array([40.5       ,  0.        , 21.75      ,  3.37262013]), array([30.4 ,  0.  , 43.  ,  2.55]), array([4.00389864e+01, 6.40149834e-02, 1.00045181e+02, 1.42108547e+0

In [4]:
Cez = 1

k2 = 0.1 
k3 = 2
k4 = 0.3
k5 = 0.4
KS = 5.0
KP = 0.9
KN = 4.0
kmenos4 = 0.01
P = np.zeros(8)
P[0]   = k2 
P[1]   = k3    
P[2]   = k4      
P[3]   = k5      
P[4]   = KS     
P[5]   = KP      
P[6]   = KN      
P[7]   = kmenos4

Np = len(P)


In [5]:
def enzymic_amox(t,y, 
k2,
k3,
k4,
k5,
KS,
KP,
KN,
kmenos4,
):
    Cez = 1
    CAB = y[0]
    CAN = y[1]
    CNH = y[2]
    CAOH = y[3]

    RP = (Cez)/(k3 * KN + k4 * CNH + k5 * CNH) * ((k2 * k4 * CAB * CNH)/ (KS) - (kmenos4 * CAN * (k3 * KN + k5 * CNH)/KP))
    RB = Cez * (k3 * KN + k5 * CNH) / (k3 * KN + k4 * CNH + k5 * CNH) * ((k2 * CAB) / KS - (kmenos4 * CAN) / KP)
    RS = -(RP + RB)
    RNu = -(RP)

    dy = np.zeros(4)

    dy[0] =  RS       
    dy[1] =  RP
    dy[2] =  -(RP)  
    dy[3] = RB      
    
    return np.array(dy)   

In [6]:
def ode15s_amox(P, CI, t):
    return scipy.integrate.solve_ivp(enzymic_amox,t_span=(t[0],t[-1]),t_eval=t,y0=CI,method='BDF',args=P).y.T

In [7]:
import scipy.integrate


def ode15s_amox(P, CI, t):
    try:
        sol = scipy.integrate.solve_ivp(
            enzymic_amox, 
            t_span=(t[0], t[-1]), 
            t_eval=t, 
            y0=CI, 
            method='RK45', 
            args=P, 
            #atol=1e-8, 
            #tol=1e-6
        )
        if sol.status != 0:
            raise ValueError("ODE solver failed to converge")
        return sol.y.T
    except Exception as e:
        print(f"Solver failed with error: {e}")
        # Handle solver failure (e.g., return NaNs or retry with different parameters)
        return np.full((len(CI), len(t)), np.nan).T
    

In [14]:
def mcmc(P,priori,N=5000,ode_solver=ode15s_amox,status=False):

    Np = len(P)
    # Extraindo dados
    exps = []
    times = []
    
    for i in range(len(CIs)):
        t = data_frames[i].loc[:,'apa_t'].dropna().to_numpy()

        exp = np.zeros((len(t),4))
        
        exp[:,0]  = data_frames[i].loc[:,'ester_mm'].dropna().to_numpy()
        exp[:,1]  = data_frames[i].loc[:,'amox_mm'].dropna().to_numpy()
        exp[:,2] = data_frames[i].loc[:,'apa_mm'].dropna().to_numpy()
        exp[:,3]  = data_frames[i].loc[:,'aoh_mm'].dropna().to_numpy()
        exps.append(exp)

        times.append(t)
    Cez = 1
    refs = []
    desvio = np.zeros((4,len(CIs)))

    for i in range(len(CIs)):
        t = times[i]
        ref = np.zeros((len(t),4))
        Y = ode_solver(P,CIs[i],t)

        ref[:,0]   =  Y[:,0]      
        ref[:,1]   =  Y[:,1]    
        ref[:,2]   =  Y[:,2]      
        ref[:,3]  =   Y[:,3]

        refs.append(ref)

        desvio[0,i] =  0.1*max(ref[:,0]) 
        desvio[1,i] =  0.1*max(ref[:,1])
        desvio[2,i] =  0.1*max(ref[:,2])
        desvio[3,i] =  0.1*max(ref[:,3])
    
    waux      = 6e-3 
    media_g   = 1       #Média gaussiana
    desviop_g = 0.6     #Desvio dos parâmetros ao utilizar priori gaussian

    estimate = [i for i in range(10)]
    Nfix = len(estimate)

    p_ref = P                   
    p_old = P                     

    media_MCMC = media_g*P
    desvio_P   = desviop_g*P

    w = np.ones((1,Np))   
    w = w*waux  

    parametro_exato = (p_ref*np.ones((N,1))).T
    
    aceitacao    = np.zeros((1,N))  
    cadeia       = np.zeros((Np,N)) 
    conv_likeli  = np.zeros((1,N))
    k=0
    lk_old = 0
        
    for i in range(len(CIs)):
        Lk_1 = np.dot((exps[i][:,0] - refs[i][:,0]),(exps[i][:,0] - refs[i][:,0]).T) / (desvio[0,i]**2)     
        Lk_2 = np.dot((exps[i][:,1] - refs[i][:,1]),(exps[i][:,1] - refs[i][:,1]).T) / (desvio[1,i]**2)      
        Lk_3 = np.dot((exps[i][:,2] - refs[i][:,2]),(exps[i][:,2] - refs[i][:,2]).T) / (desvio[2,i]**2)       
        Lk_4 = np.dot((exps[i][:,3] - refs[i][:,3]),(exps[i][:,3] - refs[i][:,3]).T) / (desvio[3,i]**2)  
        lk_partial = Lk_1 + Lk_2 + Lk_3 + Lk_4
        lk_old += lk_partial
    

    prior_old = np.sum((((p_old - media_MCMC))/((desvio_P)**2)))
    cadeia[:,0] = p_old
    
    # Contador para verificar a aceitação
    for i in range(0, N):  # Python index starts at 0, MATLAB at 1
        progress = (i / N) * 100
        if status:
            print(progress,end='\r')
        # New parameter vector
        P_new = p_old + w * np.random.randn(Np) * p_old
        #print('p_new: ', P_new)
        # Simulate using ODE solver

        chain_ref = []
        for j in range(len(CIs)):
            t = times[j]
            ref = np.zeros((len(t),4))

            Y = ode_solver(P_new[0],CIs[j],t)
            ref[:,0]  =  Y[:,0]      
            ref[:,1]  =  Y[:,1]    
            ref[:,2]  =  Y[:,2]      
            ref[:,3] =   Y[:,3]
            chain_ref.append(ref)


        # Calculate new prior
        Prior_new = np.sum(((P_new[0] - media_MCMC) / desvio_P) ** 2)

        Lk_new = 0
        # Likelihood calculations
        for j in range(len(CIs)):
            Lk_1 = np.dot((exps[j][:,0] - chain_ref[j][:,0]),(exps[j][:,0] - chain_ref[j][:,0]).T) / (desvio[0,j]**2)     
            Lk_2 = np.dot((exps[j][:,1] - chain_ref[j][:,1]),(exps[j][:,1] - chain_ref[j][:,1]).T) / (desvio[1,j]**2)      
            Lk_3 = np.dot((exps[j][:,2] - chain_ref[j][:,2]),(exps[j][:,2] - chain_ref[j][:,2]).T) / (desvio[2,j]**2)       
            Lk_4 = np.dot((exps[j][:,3] - chain_ref[j][:,3]),(exps[j][:,3] - chain_ref[j][:,3]).T) / (desvio[3,j]**2)  
            lk_partial = Lk_1 + Lk_2 + Lk_3 + Lk_4
            Lk_new += lk_partial 
    
        # MCMC acceptance check
        if np.log(random.random()) < (-0.5 * (Lk_new + Prior_new - lk_old - prior_old)):
            p_old = P_new
            lk_old = Lk_new
            prior_old = Prior_new
            k += 1
        
        # Store results
        aceitacao[0,i] = k
        cadeia[:, i] = p_old
        conv_likeli[0,i] = lk_old + prior_old
    print("aceitacao: ", k/N, 'k: ', k)
    fig,ax = plt.subplots(2,4,figsize=[20,8])
    ax = ax.flatten()
    labels = ['k2',
'k3',
'k4',
'k5',
'KS',
'KP',
'KN',
'kmenos4']
    for idx,g in enumerate(ax):
        g.plot(cadeia[idx,:])    
        g.set_title(labels[idx])
        g.set_ylabel("Value")
        g.set_xlabel("Iteration")
    fig.savefig(f"./chains/multi_{priori}_{N}.png",dpi=400)
    plt.close()    

    aq = int(0.9 * N)  # Starting index for the burn-in period
    IC = 0.99          # Confidence interval level

    # Confidence interval bounds
    xaux = (1 - IC) / 2
    IC_inf = xaux
    IC_sup = 1 - xaux
    estimate = [i for i in range(Np)]

    amostra = N - aq + 1
    amostras = [] # [CIs, Nics, t, subs]


    output_par = np.zeros((amostra,Np))
    for i in range(len(CIs)):
        t = times[i]
        inst_amostra = np.zeros((len(list(range(aq, N + 1))),len(t),4))

        for idx,j in enumerate(range(aq, N + 1)):
            ii = j - aq
            paux = cadeia[:, j - 1]
            output_par[ii,:] = paux
            
            Y = ode_solver(paux,CIs[i],t)
            inst_amostra[idx,:,0]  = Y[:,0]
            inst_amostra[idx,:,1] = Y[:,1]
            inst_amostra[idx,:,2] = Y[:,2]
            inst_amostra[idx,:,3] = Y[:,3]
        
        amostras.append(inst_amostra)


    AB_media = []
    AN_media = []
    NH_media = []
    AOH_media =[]
    AB_sup = []
    AN_sup = []
    NH_sup = []
    AOH_sup =[]
    AB_inf = []
    AN_inf = []
    NH_inf = []
    AOH_inf =  []   

    for i in range(len(CIs)):
        t = times[i]
        amostra = amostras[i]
        
        AB_mean = np.zeros(len(t))
        AB_inferior = np.zeros(len(t))
        AB_superior = np.zeros(len(t))
        NH_mean = np.zeros(len(t))
        NH_inferior = np.zeros(len(t))
        NH_superior = np.zeros(len(t))

        for j in range(len(t)):    
            AB_mean[j] = np.mean(amostra[:,j,0])
            y = np.percentile(amostra[:, j,0], [IC_inf * 100, IC_sup * 100])
            AB_inferior[j] = y[0]
            AB_superior[j] = y[1]

            NH_mean[j] = np.mean(amostra[:, j,2])
            y = np.percentile(amostra[:, j,2], [IC_inf * 100, IC_sup * 100])
            NH_inferior[j] = y[0]
            NH_superior[j] = y[1]
        
        AN_mean = np.zeros(len(t))
        AN_inferior = np.zeros(len(t))
        AN_superior = np.zeros(len(t))
        AOH_mean = np.zeros(len(t))
        AOH_inferior = np.zeros(len(t))
        AOH_superior = np.zeros(len(t))
        
        for j in range(len(t)):
            AN_mean[j] = np.mean(amostra[:, j, 1])
            y = np.percentile(amostra[:, j,1], [IC_inf * 100, IC_sup * 100])
            AN_inferior[j] = y[0]
            AN_superior[j] = y[1]

            AOH_mean[j] = np.mean(amostra[:, j,3])
            y = np.percentile(amostra[:, j,3], [IC_inf * 100, IC_sup * 100])
            AOH_inferior[j] = y[0]
            AOH_superior[j] = y[1]

        AB_media.append(AB_mean)
        NH_media.append(NH_mean)
        AB_inf.append(AB_inferior)
        AB_sup.append(AB_superior)
        NH_inf.append(NH_inferior)
        NH_sup.append(NH_superior)
        AN_media.append(AN_mean)
        AOH_media.append(AOH_mean)
        AN_inf.append(AN_inferior)
        AN_sup.append(AN_superior)
        AOH_inf.append(AOH_inferior)
        AOH_sup.append(AOH_superior)

    # individual plot
    # Ensure 'results' directory exists
    results_dir = 'results_mcmc/'

    if not os.path.exists(results_dir):
        os.makedirs(results_dir)
    for i in range(len(CIs)):
        names = ['POH-PGME','Amoxicillin','6-APA','POHPG']
        t = times[i]
        style = ['or','og','ob','ok']
        mstyle = ['-r','-g','-b','-k']
        medidas = [
            [AB_media[i], AB_inf[i], AB_sup[i]],
            [AN_media[i], AN_inf[i], AN_sup[i]],
            [NH_media[i], NH_inf[i], NH_sup[i]],
            [AOH_media[i],AOH_inf[i],AOH_sup[i]]
        ]
        fig,ax = plt.subplots(2,2,figsize=(10,10))
        for idx,axis in enumerate(ax.flatten()):
            sname = names[idx]
            medida = medidas[idx]
            
            axis.plot(t, exps[i][:,idx], style[idx], label=sname)
            axis.plot(t, medida[0], mstyle[idx])
            axis.plot(t, medida[1], '--r',linewidth=0.5)
            axis.plot(t, medida[2], '--r',linewidth=0.5) 
            axis.set_xlabel('Tempo (min)')
            axis.set_ylabel(f'Concentração de {sname} (mm)')  # Use f-string for proper string formatting
            axis.legend()  # Adjust legend position
        fig.savefig(os.path.join(results_dir, f"{i}_multi_{priori}_{N}_{file_list[i]}_mcmc.png"), dpi=300)
        plt.close()  # Close the figure to release memory
    return output_par,k/N
        


In [ ]:
p1k = mcmc(P=P,priori='donald',N=100000,status=True)

In [15]:
p2k = mcmc(P=P,priori='donald',N=200000,status=True)

aceitacao:  0.49 k:  49


(array([[0.09035903, 2.12223599, 0.29474951, 0.40643681, 5.9204636 ,
         0.88916506, 3.7795099 , 0.01056602],
        [0.09080204, 2.10631557, 0.29427064, 0.40627059, 5.96667019,
         0.89052923, 3.79005801, 0.01061014],
        [0.09080204, 2.10631557, 0.29427064, 0.40627059, 5.96667019,
         0.89052923, 3.79005801, 0.01061014],
        [0.09065665, 2.09077858, 0.2939206 , 0.40610185, 5.97746716,
         0.88109922, 3.76076942, 0.0106506 ],
        [0.09065665, 2.09077858, 0.2939206 , 0.40610185, 5.97746716,
         0.88109922, 3.76076942, 0.0106506 ],
        [0.09036558, 2.08503362, 0.29499208, 0.40020741, 5.98538513,
         0.88494094, 3.76345214, 0.01056966],
        [0.09036558, 2.08503362, 0.29499208, 0.40020741, 5.98538513,
         0.88494094, 3.76345214, 0.01056966],
        [0.09036558, 2.08503362, 0.29499208, 0.40020741, 5.98538513,
         0.88494094, 3.76345214, 0.01056966],
        [0.09044416, 2.09736676, 0.29269942, 0.39878208, 6.02081278,
         0.

In [16]:
def model_eval(x_model, t_model, val_x, val_t):
    fig, axis = plt.subplots(2, 2, figsize=[6, 6])
    labels = ['POH-PGME','Amoxicillin','6-APA','POHPG']
    mlabels = ['POH-PGME model','Amoxicillin model','6-APA model','POHPG model']
    style = ['.r','.g','.b','.k']
    mstyle = ['-k','-k','-k','-k']    
    for idx,ax in enumerate(axis.flatten()):
        ax.plot(val_t, val_x[:, idx], style[idx], label=labels[idx],linewidth=0.5,markersize=5)
        ax.plot(t_model, x_model[:, idx], mstyle[idx], label=mlabels[idx],linewidth=0.9,markersize=3)
        ax.set_title(f'{labels[idx]}')
        ax.set_ylim([val_x[:, idx].min()-5, val_x[:, idx].max()+5])
        ax.set_xlim([0, val_t[-1]+5])
        ax.legend()

In [17]:
def mass_deviation(model_output, experimental_data):
    
    exp_mass = np.sum(model_output, axis=1)    
    model_mass = np.sum(experimental_data, axis=1)

    mass_deviation = np.sum(np.abs(exp_mass - model_mass))
    return mass_deviation

In [19]:
def eval_all(P_new,less_bad=False):
    x_names = ['ester_mm', 'amox_mm', 'apa_mm', 'aoh_mm']

    total_error = 0
    if less_bad:
        surface_error = np.zeros((len(data_frames),3)) # columns = mse error, ester init, apa init 
    else:
        surface_error = np.zeros((len(data_frames),3)) # columns = mse error, ester init, apa init 
    
    total_mass_error = 0
    for idx,df in enumerate(data_frames):
        x_target = df.loc[:,x_names].to_numpy()
        t_target = df['apa_t'].to_numpy()
    
        ic = x_target[0]
        

        surface_error[idx,1] = ic[0] # ester
        surface_error[idx,2] = ic[2] # apa
        

        if ((ic[0] == 20 and ic[2] == 60) or (ic[0] == 80 and ic[2] == 60)) and (less_bad):
            surface_error[idx,0] = 0
            continue

        t_model = np.linspace(t_target[0],t_target[-1],40)
        
        sol = ode15s_amox(P_new,ic,t_model)
        sol_val = ode15s_amox(P_new,ic,t_target).T

        error = root_mean_squared_error(x_target.T, sol_val)
        mass_error = mass_deviation(sol_val,x_target.T)
        r2 = r2_score(x_target.T,sol_val)

        surface_error[idx,0] = error

        total_error += error
        total_mass_error += mass_error
        
        print(f'Error {ic}: {error}\tmass: {mass_error}\tr2: {r2}')

        x_model = sol.T
        model_eval(x_model.T,t_model,x_target,t_target)
    surfacedf = pd.DataFrame(surface_error,columns=['RMSE','Ester','APA'])
    
    plt.figure(figsize=(6,6))
    sns.scatterplot(data=surfacedf, x='Ester', y='APA', size='RMSE', hue='RMSE', sizes=(20, 200),legend='full')
    plt.legend(title='Error Value', loc='upper right', bbox_to_anchor=(-0.15, 1))
    #sns.heatmap(surfacedf,annot=True,fmt=".3f")
    print(f'Total error: {total_error/len(data_frames)}\tmass error:{total_mass_error/len(data_frames)}')

In [20]:
labels = ['k2',
'k3',
'k4',
'k5',
'KS',
'KP',
'KN',
'kmenos4']

labels_df = ['k2',
'k3',
'k4',
'k5',
'KS',
'KP',
'KN',
'kmenos4','aceitacao']


# all_P = [luci_P,lucas_P,process_P,optimized_P]
#names = ['multi_luci','multi_lucas','multi_process','multi_bestnm']
Ns = [100000,200000]

In [24]:
CIs = []

def import_dfs_less_bad():
    # Path to the folder containing CSV files
    folder_path = '../dados/'

    # Get a list of all CSV files in the folder
    csv_files = glob.glob(os.path.join(folder_path, '*.csv'))
    # Initialize an empty list to hold dataframes
    data_frames = []

    # Read the first CSV file to establish the schema
    schema = ['ester_mm', 'amox_mm', 'apa_mm', 'aoh_mm','apa_t']

    csv_files.remove("../dados\\25.45nh60ab20.csv")
    csv_files.remove("../dados\\nh60ab80.csv")
    print(csv_files)
    # Load each remaining CSV file, reorder columns, and append to the list
    for file in csv_files:
        df = pd.read_csv(file)  

        df = df[schema]  # Reorder columns to match the schema
        data_frames.append(df)
        CIs.append(df.iloc[0,:4].to_numpy())
    csv_files = [name.split('\\')[-1].rstrip('.csv') for name in csv_files]


    return data_frames,csv_files 
data_frames,file_list = import_dfs_less_bad()
print(data_frames[0].columns)
print(file_list)
print(CIs)

['../dados\\25.46nh30ab30.csv', '../dados\\25.47nh5ab80.csv', '../dados\\5.100nh40ab80.csv', '../dados\\5.102nh20ab40.csv', '../dados\\5.103nh40ab30.csv', '../dados\\5.106nh100ab40.csv', '../dados\\5.107nh55ab55.csv', '../dados\\5.109nh5ab55.csv', '../dados\\5.110nh10ab55.csv', '../dados\\5.112nh78ab35.csv', '../dados\\5.48nh20ab80.csv', '../dados\\5.80nh12ab40.csv', '../dados\\5.81nh5ab30.csv']
Index(['ester_mm', 'amox_mm', 'apa_mm', 'aoh_mm', 'apa_t'], dtype='object')
['25.46nh30ab30', '25.47nh5ab80', '5.100nh40ab80', '5.102nh20ab40', '5.103nh40ab30', '5.106nh100ab40', '5.107nh55ab55', '5.109nh5ab55', '5.110nh10ab55', '5.112nh78ab35', '5.48nh20ab80', '5.80nh12ab40', '5.81nh5ab30']
[array([30.,  0., 30.,  0.]), array([80.,  0.,  5.,  0.]), array([80.,  0., 40.,  0.]), array([40.5       ,  0.        , 21.75      ,  3.37262013]), array([30.4 ,  0.  , 43.  ,  2.55]), array([4.00389864e+01, 6.40149834e-02, 1.00045181e+02, 1.42108547e+00]), array([55.      ,  0.      , 55.      ,  0.623377

In [29]:
def eval_performance(P,plot=False,less_bad=False):
    total_error = 0
    x_names = ['ester_mm', 'amox_mm', 'apa_mm', 'aoh_mm']
    for exp_name,df_exp in zip(file_list,data_frames):
        if (exp_name == '25.45nh60ab20' or exp_name == 'nh60ab80') and less_bad:
            continue
        x_target = df_exp.loc[:,x_names].to_numpy()
        t_target = df_exp['apa_t'].to_numpy()
        
        ic = x_target[0]
        t_model = np.linspace(t_target[0],t_target[-1],len(t_target))
        
        sol_val = ode15s_amox(P,ic,t_target).T
        x_model = sol_val
        error = root_mean_squared_error(x_target.T, sol_val)

        total_error += error
        if plot:
            print(f'Error {exp_name}: {error}')
        
            model_eval(x_model,t_target,x_target,t_target)
    if plot:
        print('total error: ',total_error)
    return total_error
all_par_df.index

32.48330172706005